In [1]:
import gc
import torch
import torch.optim as optim
from torchinfo import summary

from KSI_models import KSI, ModifiedKSI, LSTMattn
from KSI_utils import load_KSI_data, train_model, test_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
n_embedding = 100
n_hidden = 100 # 200 in paper, but too intensive for my machine
batch_size = 32
n_epochs = 25
save = True
profile = False
model_type = 'LSTMattn'
early_stopping = 5

In [3]:
dir = 'data/original/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [4]:
# note_lengths = []
# for data in train_dataloader:
#     n, _, _ = data
#     note_lengths.append(n.shape[1])
# avg_note_size = np.round(np.array(note_lengths).mean()).astype(int)

avg_note_size = 2455

In [5]:
base_model = LSTMattn(n_words, n_wiki, n_embedding, n_hidden, batch_size)
base_model = base_model.to(DEVICE)
base_summary = summary(base_model, [(batch_size, avg_note_size), 
                                    (batch_size, n_vocab)], 
                       dtypes=[torch.int, torch.float])

base_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTMattn                                 --                        --
├─Embedding: 1-1                         [32, 2455, 100]           4,796,200
├─Dropout: 1-2                           [2455, 32, 100]           --
├─LSTM: 1-3                              [2455, 32, 100]           80,800
├─Linear: 1-4                            --                        34,400
├─Linear: 1-5                            --                        34,744
Total params: 4,946,144
Trainable params: 4,946,144
Non-trainable params: 0
Total mult-adds (G): 6.50
Input size (MB): 1.87
Forward/backward pass size (MB): 125.70
Params size (MB): 19.78
Estimated Total Size (MB): 147.35

In [6]:
optimizer = optim.Adam(base_model.parameters())
prof_base = train_model(base_model, 
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        n_epochs=n_epochs,
                        profile=profile, 
                        log_path=f'./log/{model_type}',
                        device=DEVICE,
                        init_hidden=True,
                        early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.4405, Val Recall@10: 0.4460, Train Micro F1: 0.0000, Val Micro F1: 0.0000, Train Macro F1: 0.0000, Val Macro F1: 0.0000, Train Micro AUC: 0.9269, Val Micro AUC: 0.9123, Train Macro AUC: 0.5821, Val Macro AUC: 0.5883
Epoch: 002, Train Recall@10: 0.5541, Val Recall@10: 0.5566, Train Micro F1: 0.4543, Val Micro F1: 0.4571, Train Macro F1: 0.0207, Val Macro F1: 0.0252, Train Micro AUC: 0.9415, Val Micro AUC: 0.9295, Train Macro AUC: 0.6246, Val Macro AUC: 0.6363
Epoch: 003, Train Recall@10: 0.6379, Val Recall@10: 0.6377, Train Micro F1: 0.5660, Val Micro F1: 0.5633, Train Macro F1: 0.0509, Val Macro F1: 0.0610, Train Micro AUC: 0.9536, Val Micro AUC: 0.9435, Train Macro AUC: 0.6631, Val Macro AUC: 0.6796
Epoch: 004, Train Recall@10: 0.7006, Val Recall@10: 0.6949, Train Micro F1: 0.6051, Val Micro F1: 0.5996, Train Macro F1: 0.0719, Val Macro F1: 0.0858, Train Micro AUC: 0.9614, Val Micro AUC: 0.9526, Train Macro AUC: 0.6839, Val Macro AUC: 0.7050
Epoch: 005, 

In [7]:
if save:
    torch.save(base_model, f'{dir}{model_type}_model.pt')
if profile:
    print(prof_base.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [8]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_base = test_model(base_model, 
                                                                                                   test_dataloader, 
                                                                                                   wikivec,
                                                                                                   device=DEVICE,
                                                                                                   init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del base_model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.8242, Test Micro F1: 0.6848, Test Macro F1: 0.2594, Test Micro AUC: 0.9805, Test Macro AUC: 0.8545


In [9]:
ksi = KSI(n_embedding, n_vocab)
ksi.to(DEVICE)
model = LSTMattn(n_words, n_wiki, n_embedding, n_hidden, ksi=ksi)
model = model.to(DEVICE)
ksi_summary = summary(model, [(batch_size, avg_note_size), 
                              (batch_size, n_vocab),
                              (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

ksi_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTMattn                                 --                        --
├─KSI: 1-1                               --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [2455, 32, 100]           --
├─LSTM: 1-4                              [2455, 32, 100]           80,800
├─KSI: 1-1                               --                        --
│    └─Linear: 2-4                       [32, 344, 100]            1,217,400
│    └─Linear: 2-5                       [32, 344, 100]            10,100
│    └─Linear: 2-6                       [32, 344, 1]              101
Total params: 6,104,601
Trainable p

In [10]:
optimizer = optim.Adam(model.parameters())
prof_ksi = train_model(model, 
                       train_dataloader=train_dataloader,
                       val_dataloader=val_dataloader,
                       wikivec=wikivec,
                       optimizer=optimizer,
                       n_epochs=n_epochs, 
                       profile=profile, 
                       log_path=f'./log/{model_type}_KSI',
                       device=DEVICE,
                       init_hidden=True,
                       early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.6576, Val Recall@10: 0.6528, Train Micro F1: 0.4033, Val Micro F1: 0.3975, Train Macro F1: 0.0596, Val Macro F1: 0.0709, Train Micro AUC: 0.9669, Val Micro AUC: 0.9588, Train Macro AUC: 0.8290, Val Macro AUC: 0.8168
Epoch: 002, Train Recall@10: 0.7019, Val Recall@10: 0.6958, Train Micro F1: 0.4814, Val Micro F1: 0.4733, Train Macro F1: 0.0872, Val Macro F1: 0.0991, Train Micro AUC: 0.9720, Val Micro AUC: 0.9636, Train Macro AUC: 0.8622, Val Macro AUC: 0.8375
Epoch: 003, Train Recall@10: 0.7398, Val Recall@10: 0.7277, Train Micro F1: 0.5297, Val Micro F1: 0.5138, Train Macro F1: 0.1134, Val Macro F1: 0.1225, Train Micro AUC: 0.9764, Val Micro AUC: 0.9680, Train Macro AUC: 0.8833, Val Macro AUC: 0.8478
Epoch: 004, Train Recall@10: 0.7664, Val Recall@10: 0.7507, Train Micro F1: 0.5782, Val Micro F1: 0.5582, Train Macro F1: 0.1451, Val Macro F1: 0.1492, Train Micro AUC: 0.9793, Val Micro AUC: 0.9704, Train Macro AUC: 0.8978, Val Macro AUC: 0.8553
Epoch: 005, 

In [11]:
if save:
    torch.save(model, f'{dir}{model_type}_KSI_model.pt')
if profile:
    print(prof_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [12]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_ksi = test_model(model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE,
                                                                                                  init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7755, Test Micro F1: 0.6124, Test Macro F1: 0.2103, Test Micro AUC: 0.9745, Test Macro AUC: 0.8801


In [13]:
# run modified KSI using frequency vectors rather than binary vectors
dir = 'data/original_freqs/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [14]:
mod_ksi = ModifiedKSI(n_embedding, n_vocab)
mod_ksi.to(DEVICE)
mod_model = LSTMattn(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi)
mod_model = mod_model.to(DEVICE)
mod_summary = summary(mod_model, [(batch_size, avg_note_size), 
                                  (batch_size, n_vocab),
                                  (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

mod_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTMattn                                 --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [2455, 32, 100]           --
├─LSTM: 1-4                              [2455, 32, 100]           80,800
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                       [32, 344, 100]            1,217,400
│    └─Linear: 2-7              

In [15]:
optimizer = optim.Adam(mod_model.parameters())
prof_mod_ksi = train_model(mod_model, 
                           train_dataloader=train_dataloader,
                           val_dataloader=val_dataloader,
                           wikivec=wikivec,
                           optimizer=optimizer,
                           n_epochs=n_epochs, 
                           profile=profile, 
                           log_path=f'./log/{model_type}_ModifiedKSI',
                           device=DEVICE,
                           init_hidden=True,
                           early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.6862, Val Recall@10: 0.6851, Train Micro F1: 0.4554, Val Micro F1: 0.4497, Train Macro F1: 0.0518, Val Macro F1: 0.0617, Train Micro AUC: 0.9693, Val Micro AUC: 0.9628, Train Macro AUC: 0.8360, Val Macro AUC: 0.8402
Epoch: 002, Train Recall@10: 0.7156, Val Recall@10: 0.7135, Train Micro F1: 0.4894, Val Micro F1: 0.4830, Train Macro F1: 0.0976, Val Macro F1: 0.1166, Train Micro AUC: 0.9739, Val Micro AUC: 0.9679, Train Macro AUC: 0.8599, Val Macro AUC: 0.8598
Epoch: 003, Train Recall@10: 0.7322, Val Recall@10: 0.7269, Train Micro F1: 0.5043, Val Micro F1: 0.4980, Train Macro F1: 0.1174, Val Macro F1: 0.1403, Train Micro AUC: 0.9759, Val Micro AUC: 0.9701, Train Macro AUC: 0.8692, Val Macro AUC: 0.8652
Epoch: 004, Train Recall@10: 0.7597, Val Recall@10: 0.7556, Train Micro F1: 0.5603, Val Micro F1: 0.5520, Train Macro F1: 0.1394, Val Macro F1: 0.1483, Train Micro AUC: 0.9783, Val Micro AUC: 0.9727, Train Macro AUC: 0.8816, Val Macro AUC: 0.8738
Epoch: 005, 

In [16]:
if save:
    torch.save(mod_model, f'{dir}{model_type}_ModifiedKSI_model.pt')
if profile:
    print(prof_mod_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [17]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(mod_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE,
                                                                                                  init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del mod_model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.8119, Test Micro F1: 0.6483, Test Macro F1: 0.2476, Test Micro AUC: 0.9812, Test Macro AUC: 0.9063


In [18]:
# run modified KSI using tfidf vectors rather than binary vectors
dir = 'data/original_tfidf/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [19]:
mod_ksi2 = ModifiedKSI(n_embedding, n_vocab)
mod_ksi2.to(DEVICE)
tfidf_model = LSTMattn(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi2)
tfidf_model = tfidf_model.to(DEVICE)
tfidf_summary = summary(tfidf_model, [(batch_size, avg_note_size), 
                                      (batch_size, n_vocab),
                                      (n_wiki, n_vocab)], 
                        dtypes=[torch.int, torch.float, torch.float])

tfidf_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTMattn                                 --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [2455, 32, 100]           --
├─LSTM: 1-4                              [2455, 32, 100]           80,800
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                       [32, 344, 100]            1,217,400
│    └─Linear: 2-7              

In [20]:
optimizer = optim.Adam(tfidf_model.parameters())
prof_tfidf_ksi = train_model(tfidf_model, 
                             train_dataloader=train_dataloader,
                             val_dataloader=val_dataloader,
                             wikivec=wikivec,
                             optimizer=optimizer,
                             n_epochs=n_epochs, 
                             profile=profile, 
                             log_path=f'./log/{model_type}_ModifiedKSI_tfidf',
                             device=DEVICE,
                             init_hidden=True,
                             early_stopping=early_stopping)

Epoch: 001, Train Recall@10: 0.6786, Val Recall@10: 0.6752, Train Micro F1: 0.4677, Val Micro F1: 0.4649, Train Macro F1: 0.0703, Val Macro F1: 0.0857, Train Micro AUC: 0.9691, Val Micro AUC: 0.9628, Train Macro AUC: 0.8340, Val Macro AUC: 0.8403
Epoch: 002, Train Recall@10: 0.7019, Val Recall@10: 0.6985, Train Micro F1: 0.4768, Val Micro F1: 0.4731, Train Macro F1: 0.0923, Val Macro F1: 0.1065, Train Micro AUC: 0.9723, Val Micro AUC: 0.9662, Train Macro AUC: 0.8561, Val Macro AUC: 0.8549
Epoch: 003, Train Recall@10: 0.7151, Val Recall@10: 0.7138, Train Micro F1: 0.4888, Val Micro F1: 0.4825, Train Macro F1: 0.1151, Val Macro F1: 0.1228, Train Micro AUC: 0.9742, Val Micro AUC: 0.9681, Train Macro AUC: 0.8675, Val Macro AUC: 0.8641
Epoch: 004, Train Recall@10: 0.7256, Val Recall@10: 0.7228, Train Micro F1: 0.5020, Val Micro F1: 0.4953, Train Macro F1: 0.1327, Val Macro F1: 0.1420, Train Micro AUC: 0.9759, Val Micro AUC: 0.9697, Train Macro AUC: 0.8771, Val Macro AUC: 0.8693
Epoch: 005, 

In [21]:
if save:
    torch.save(tfidf_model, f'{dir}{model_type}_ModifiedKSI_tfidf_model.pt')
if profile:
    print(prof_tfidf_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [22]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(tfidf_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE,
                                                                                                  init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del tfidf_model
gc.collect()
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7973, Test Micro F1: 0.6257, Test Macro F1: 0.2481, Test Micro AUC: 0.9795, Test Macro AUC: 0.8985
